# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one (or multiple)
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using AtomsBuilder
using DFTK
using LinearAlgebra
using Unitful
using UnitfulAtomic
using PseudoPotentialData

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    # Use AtomsBuilder to setup aluminium cubic unit cell (4 Al atoms)
    # with provided lattice constant, see AtomsBase integration for details.
    unit_cell = bulk(:Al; a=7.65339u"bohr", cubic=true)
    supercell = unit_cell * (repeat, 1, 1)  # Make a supercell

    # Select standard pseudodojo pseudopotentials, construct an LDA model, discretize
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    pseudopotentials = PseudoFamily("dojo.nc.sr.lda.v0_4_1.standard.upf")
    model = model_DFT(supercell; pseudopotentials, functionals=LDA(),
                      temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As expected we obtain the unit cell for `repeat=1`:

In [2]:
aluminium_setup(1)

PlaneWaveBasis discretization:
    architecture         : DFTK.CPU()
    num. mpi processes   : 1
    num. julia threads   : 1
    num. DFTK  threads   : 1
    num. blas  threads   : 2
    num. fft   threads   : 1

    Ecut                 : 7.0 Ha
    fft_size             : (24, 24, 24), 13824 total points
    kgrid                : MonkhorstPack([2, 2, 2])
    num.   red. kpoints  : 8
    num. irred. kpoints  : 8

    Discretized Model(lda_x+lda_c_pw, 3D):
        lattice (in Bohr)    : [7.65339   , 0         , 0         ]
                               [0         , 7.65339   , 0         ]
                               [0         , 0         , 7.65339   ]
        unit cell volume     : 448.29 Bohr³
    
        atoms                : Al₄
        pseudopot. family    : PseudoFamily("dojo.nc.sr.lda.v0_4_1.standard.upf")
    
        num. electrons       : 12
        spin polarization    : none
        temperature          : 0.001 Ha
        smearing             : DFTK.Smearing.FermiDi

and 5-fold as large supercell with `repeat=5`:

In [3]:
aluminium_setup(5)

PlaneWaveBasis discretization:
    architecture         : DFTK.CPU()
    num. mpi processes   : 1
    num. julia threads   : 1
    num. DFTK  threads   : 1
    num. blas  threads   : 2
    num. fft   threads   : 1

    Ecut                 : 7.0 Ha
    fft_size             : (96, 24, 24), 55296 total points
    kgrid                : MonkhorstPack([2, 2, 2])
    num.   red. kpoints  : 8
    num. irred. kpoints  : 8

    Discretized Model(lda_x+lda_c_pw, 3D):
        lattice (in Bohr)    : [38.267    , 0         , 0         ]
                               [0         , 7.65339   , 0         ]
                               [0         , 0         , 7.65339   ]
        unit cell volume     : 2241.5 Bohr³
    
        atoms                : Al₂₀
        pseudopot. family    : PseudoFamily("dojo.nc.sr.lda.v0_4_1.standard.upf")
    
        num. electrons       : 60
        spin polarization    : none
        temperature          : 0.001 Ha
        smearing             : DFTK.Smearing.FermiD

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [4]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -9.355273078989                   -1.10    5.9    162ms
  2   -9.356791065974       -2.82       -1.43    1.0   78.5ms
  3   -9.357060518710       -3.57       -2.76    2.4   94.9ms
  4   -9.357110574065       -4.30       -3.00   10.0    220ms
  5   -9.357111008870       -6.36       -3.12    1.0   78.4ms
  6   -9.357111266969       -6.59       -3.26    1.1   76.8ms
  7   -9.357111417776       -6.82       -3.42    2.1    130ms
  8   -9.357111476567       -7.23       -3.60    1.0    508ms
  9   -9.357111502636       -7.58       -3.89    1.0   76.7ms
 10   -9.357111507518       -8.31       -4.21    1.0   76.8ms


In [5]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -18.74768962003                   -0.97    6.2    383ms
┌ Warning: Eigensolver not converged
│   n_iter =
│    8-element Vector{Int64}:
│     1
│     1
│     1
│     1
│     1
│     6
│     1
│     1
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:76
  2   -18.75468724574       -2.16       -1.34    1.6    247ms
┌ Warning: Eigensolver not converged
│   n_iter =
│    8-element Vector{Int64}:
│     5
│     2
│     3
│     9
│     3
│     4
│     1
│     7
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:76
  3   -18.79231054232       -1.42       -1.91    4.2    306ms
┌ Warning: Eigensolver not converged
│   n_iter =
│    8-element Vector{Int64}:
│      2
│     12
│      2
│      5
│      1
│      9
│      1
│      3
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:76
  4   -18.79257299524       -3.58       -2.15  

In [6]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -37.54677282249                   -0.84    8.8    2.16s
  2   -37.55810287473       -1.95       -1.23    3.4    852ms
  3   -37.55994438103       -2.73       -2.50    7.4    1.26s
  4   -37.56493773819       -2.30       -2.20   12.9    1.72s
  5   -37.56493882673       -5.96       -2.18    6.6    1.03s
  6   -37.56494859325       -5.01       -2.86    1.0    808ms
  7   -37.56494927647       -6.17       -3.30    1.9    1.19s
  8   -37.56494970475       -6.37       -3.64    6.4    1.25s
  9   -37.56494985271       -6.83       -4.27    4.5    985ms


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [7]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -9.355226430874                   -1.10    5.9    175ms
  2   -9.356827818806       -2.80       -1.90    1.0   75.4ms
  3   -9.357089034266       -3.58       -2.67    7.0    152ms
  4   -9.357086425745   +   -5.58       -2.55    2.4   97.8ms
  5   -9.357111227076       -4.61       -3.63    1.1   72.8ms
  6   -9.357111454289       -6.64       -3.95    5.4    131ms
  7   -9.357111507285       -7.28       -4.58    1.9   83.9ms


In [8]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -37.55015827606                   -0.84    9.2    2.13s
  2   -37.55289942307       -2.56       -1.60    2.4    698ms
  3   -29.04299031184   +    0.93       -0.60   10.4    1.70s
  4   -37.50743271186        0.93       -1.63    7.9    1.56s
  5   -37.52699334444       -1.71       -1.73    1.8    745ms
  6   -37.52548506338   +   -2.82       -1.75    2.6    905ms
  7   -37.36702626971   +   -0.80       -1.41    4.8    1.47s
  8   -37.56354555623       -0.71       -2.41    3.5    1.00s
  9   -37.56478549832       -2.91       -2.72    4.4    1.27s
 10   -37.56486400053       -4.11       -2.91    2.5    978ms
 11   -37.56491050526       -4.33       -3.07    2.1    850ms
 12   -37.56492306228       -4.90       -3.06    2.5    983ms
 13   -37.56494217689       -4.72       -3.40    1.9    830ms
 14   -37.56494798898       -5.24       -3.71    2.8    1.40s
 15   -37

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.